# **Spatial Temporal Graph Convolutional Networks for Skeleton-Based Action Recognition**





Uzaysal Zamansal Graf Evrişimsel Ağlar (ST-GCN)

<!--
<img src='https://drive.google.com/uc?id=1ZRf-NF4S0P1VwMxN2DrTFPeO4EJ5if3S' width=100%>
-->
<!--
<img src='https://github.com/machine-perception-robotics-group/MPRGDeepLearningLectureNotebook/blob/master/15_gcn/fig/03_st-gcn.png?raw=true' width=100%>
-->
<img src='https://github.com/sirakik/MPRGLecture/blob/master/15_gcn/fig/03_st-gcn.png?raw=true' width=100%>


ST-GCN' de iskelet verileri iki graf yapısı olarak ifade edilir.

- Uzaysal graf: Aynı çerçevedeki bağlantı eklemlerinin grafı
- Zaman grafiği: Bitişik çerçevelerin aynı eklemlerini birleştiren graf

Graph Convolution kullanarak uzay graflarından ve zaman graflarından öznitelikler çıkarılarak, eklemler ve zamansal değişiklikler arasındaki ilişkileri ele alınır.

# **BAŞLANGIÇ AYARLAMALARI**

---




###**Colab için Drive Bağlantısı**

In [ ]:
%cd ..
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive

In [ ]:
cd /mydrive/Doktora/UYGULAMALAR/ST-GCN/

In [ ]:
ls

###**Gerekli Kütüphaneler ve Tanımlamalar**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch #PyTorch, derin öğrenme ve tensor hesaplamaları için popüler bir kütüphanedir.
import torch.nn as nn #Bu modül, derin öğrenme modelleri oluşturmak için birçok temel yapı ve fonksiyon içerir, örneğin katmanlar ve aktivasyon fonksiyonları.
import torch.nn.functional as F #Bu modül, kayıp fonksiyonları, aktivasyon fonksiyonları ve diğer işlevselliği içeren birçok fonksiyonu barındırır.

In [ ]:
print('Use CUDA:', torch.cuda.is_available())
#device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
#Bu kod parçacığı, rastgele sayı üretiminde deterministik (tutarlı) davranış sağlamak amacıyla seed değerlerini ayarlıyor.

seed = 123 #Seed değeri, rastgele sayı üreticilerinin başlangıç noktasını belirler.
            #Aynı seed değeri kullanıldığında, rastgele sayı üreticisi her zaman aynı sayı dizisini üretir. Bu, deneylerin tekrarlanabilir olmasını sağlar.
np.random.seed(seed) # rastgele sayı üreticisinin seed değerini ayarlar.
torch.manual_seed(seed) #PyTorch için CPU üzerinde çalışan rastgele sayı üreticisinin seed değerini ayarlar.
torch.cuda.manual_seed(seed) #PyTorch için GPU üzerinde çalışan rastgele sayı üreticisinin seed değerini ayarlar.

#PyTorch'un cuDNN (CUDA Deep Neural Network) kütüphanesini deterministik bir modda kullanmasını sağlar.
#Bu, GPU üzerinde çalışan bazı işlemlerin deterministik sonuçlar üretmesini sağlar, ancak performansı bir miktar düşürebilir
torch.backends.cudnn.deterministic = True
#PyTorch'un rastgele algoritmaların nondeterministik uygulamalarını kullanmamasını sağlar. Bu, tekrarlanabilir sonuçlar sağlama amacı güder.
torch.use_deterministic_algorithms = True

#Bu ayarlamalar farklı çalıştırmalar arasında tutarlı sonuçlar üretmesine yardımcı olur, ki bu genellikle araştırma ve model doğrulama süreçlerinde önemlidir.

###**Veri Seti**

In [ ]:
train_data = np.load("../DATASETS/cs_data/train_data.npy")
print(train_data[0].shape)
print(train_data.size) # 9355 * 80 * 25 * 3
print(train_data.shape)

test_data = np.load("../DATASETS/cs_data/test_data.npy")
print(test_data[0].shape)
print(test_data.size) # 3836 * 80 * 25 * 3
print(test_data.shape)

###**Veri Kümesini Yükleme**

In [ ]:
#Yapı: Her veri 80 çerçeve için 25 eklem 3B koordinat içerir.
class Feeder(torch.utils.data.Dataset):
  def __init__(self, data_path, label_path):
      super().__init__()
      self.label = np.load(label_path)
      self.data = np.load(data_path)

  def __len__(self):
      return len(self.label)

  def __iter__(self):
      return self

  def __getitem__(self, index):
      data = np.array(self.data[index])
      label = self.label[index]

      return data, label

#get ve len testlerim
#feeder = Feeder(data_path='data/train_data.npy', label_path='data/train_label.npy')
#data, label = feeder[10]
#print(data,label)
#feeder = Feeder(data_path='data/train_data.npy', label_path='data/train_label.npy')
#len = feeder.__len__()
#print(len)

<img src='https://github.com/sirakik/MPRGLecture/blob/master/15_gcn/fig/03_skeleton.png?raw=true' width=30%>

### **Komşuluk Matrisi Oluşturma**


In [ ]:
#Elimizde sadece koordinat verisi var (düğüm özellikleri). Bağlantılar tanımlanır ve graf çizlir. Bağlantıları ifade etmek için bir komşuluk matrisi kullanılır.
class Graph():

  def __init__(self, hop_size):
    self.get_edge()
    self.hop_size = hop_size
    self.hop_dis = self.get_hop_distance(self.num_node, self.edge, hop_size=hop_size)
    self.get_adjacency()

  def __str__(self):
    return self.A #yazdırmak isteniyorsa return str(self.A) olmalıdır.

  def get_edge(self):
    self.num_node = 25
    self_link = [(i, i) for i in range(self.num_node)] # frameler arası
    neighbor_base = [(1, 2), (2, 21), (3, 21), (4, 3), (5, 21), #aynı frame 24 baglantı
                      (6, 5), (7, 6), (8, 7), (9, 21), (10, 9),
                      (11, 10), (12, 11), (13, 1), (14, 13), (15, 14),
                      (16, 15), (17, 1), (18, 17), (19, 18), (20, 19),
                      (22, 23), (23, 8), (24, 25), (25, 12)]
    neighbor_link = [(i - 1, j - 1) for (i, j) in neighbor_base] #Bu satırda, neighbor_base listesindeki tüm tuple'lardan 1 çıkarılır. Bunun nedeni, Python'da dizinlemenin 0'dan başlamasıdır. Yani, (1, 2) 0 indeksli olarak (0, 1) haline gelir.
    self.edge = self_link + neighbor_link
#self_link : [(0, 0), (1, 1), (2, 2), (3, 3), (4, 4), (5, 5), (6, 6), (7, 7), (8, 8), (9, 9), (10, 10), (11, 11), (12, 12), (13, 13), (14, 14), (15, 15), (16, 16), (17, 17), (18, 18), (19, 19), (20, 20), (21, 21), (22, 22), (23, 23), (24, 24)]
#neighbor_link: [(1, 2), (2, 21), (3, 21), (4, 3), (5, 21), (6, 5), (7, 6), (8, 7), (9, 21), (10, 9), (11, 10), (12, 11), (13, 1), (14, 13), (15, 14), (16, 15), (17, 1), (18, 17), (19, 18), (20, 19), (22, 23), (23, 8), (24, 25), (25, 12)]
#edge: [(0, 0), (1, 1), (2, 2), (3, 3), (4, 4), (5, 5), (6, 6), (7, 7), (8, 8), (9, 9), (10, 10), (11, 11), (12, 12), (13, 13), (14, 14), (15, 15), (16, 16), (17, 17), (18, 18), (19, 19), (20, 20), (21, 21), (22, 22), (23, 23), (24, 24), (0, 1), (1, 20), (2, 20), (3, 2), (4, 20), (5, 4), (6, 5), (7, 6), (8, 20), (9, 8), (10, 9), (11, 10), (12, 0), (13, 12), (14, 13), (15, 14), (16, 0), (17, 16), (18, 17), (19, 18), (21, 22), (22, 7), (23, 24), (24, 11)]


#Bu metod, her iki düğüm arasındaki minimum yol uzunluğunu (hop mesafesi) hesaplar ve bu mesafeleri bir matris şeklinde döndürür.
  def get_hop_distance(self, num_node, edge, hop_size):
    A = np.zeros((num_node, num_node))
    for i, j in edge:
        A[j, i] = 1
        A[i, j] = 1
    hop_dis = np.zeros((num_node, num_node)) + np.inf #sonsuz
    transfer_mat = [np.linalg.matrix_power(A, d) for d in range(hop_size + 1)]
    arrive_mat = (np.stack(transfer_mat) > 0)
    for d in range(hop_size, -1, -1):
        hop_dis[arrive_mat[d]] = d
    return hop_dis

  """[[ 0.  1. inf inf inf inf inf inf inf inf inf inf  1. inf inf inf  1. inf inf inf inf inf inf inf inf]
      [ 1.  0. inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf  1. inf inf inf inf]
      [inf inf  0.  1. inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf  1. inf inf inf inf]
      [inf inf  1.  0. inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf]
      [inf inf inf inf  0.  1. inf inf inf inf inf inf inf inf inf inf inf inf inf inf  1. inf inf inf inf]
      [inf inf inf inf  1.  0.  1. inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf]
      [inf inf inf inf inf  1.  0.  1. inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf]
      [inf inf inf inf inf inf  1.  0. inf inf inf inf inf inf inf inf inf inf inf inf inf inf  1. inf inf]
      [inf inf inf inf inf inf inf inf  0.  1. inf inf inf inf inf inf inf inf inf inf  1. inf inf inf inf]
      [inf inf inf inf inf inf inf inf  1.  0.  1. inf inf inf inf inf inf inf inf inf inf inf inf inf inf]
      [inf inf inf inf inf inf inf inf inf  1.  0.  1. inf inf inf inf inf inf inf inf inf inf inf inf inf]
      [inf inf inf inf inf inf inf inf inf inf  1.  0. inf inf inf inf inf inf inf inf inf inf inf inf  1.]
      ...
      [inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf  0.  1.]
      [inf inf inf inf inf inf inf inf inf inf inf  1. inf inf inf inf inf inf inf inf inf inf inf  1.  0.]]"""


#Bu metod, grafikteki düğümler arasındaki komşuluk ilişkisini temsil eden bir adjacency matrisi oluşturur. Ayrıca, matrisi normalize eder.
  def get_adjacency(self):
    valid_hop = range(0, self.hop_size + 1, 1)
    adjacency = np.zeros((self.num_node, self.num_node))
    for hop in valid_hop:
        adjacency[self.hop_dis == hop] = 1
    normalize_adjacency = self.normalize_digraph(adjacency)
    A = np.zeros((len(valid_hop), self.num_node, self.num_node))
    for i, hop in enumerate(valid_hop):
        A[i][self.hop_dis == hop] = normalize_adjacency[self.hop_dis == hop]
    self.A = A # 1. boyutta kendi kendi olan yerler 1 olur 2. boyutta ise bağlantı olan yerler 1 ama normalize edilmiş hali



#Bu metod, yönlendirilmiş bir grafik olan A matrisini normalize eder. Normalleştirme, kenar ağırlıklarının düğüm dereceleriyle ölçeklendirilmesiyle yapılır.
  def normalize_digraph(self, A):
    Dl = np.sum(A, 0)
    num_node = A.shape[0]
    Dn = np.zeros((num_node, num_node))
    for i in range(num_node):
        if Dl[i] > 0:
            Dn[i, i] = Dl[i]**(-1)
    DAD = np.dot(A, Dn)
    return DAD

###**Uzaysal Grafların Evrişimi**

İlk olarak, uzaysal grafların graf evrişimini uygulanır.

(Graph Convolutional Networks Ağlarla Düğüm Sınıflandırması).


\begin{equation}
{\bf H}_{out}=\sum_{j}{\bf\tilde D}^{-\frac{1}{2}}_j{\bf\tilde A}_j{\bf\tilde D}^{-\frac{1}{2}}_j{\bf H}_{in}{\bf W}_{j}
\end{equation}


In [ ]:
class SpatialGraphConvolution(nn.Module): #ana sınıf nn.Module’den miras
  def __init__(self, in_channels, out_channels, s_kernel_size):
    super().__init__() # nn.Module sınıfının bir örneğini oluştur
    self.s_kernel_size = s_kernel_size
    self.conv = nn.Conv2d(in_channels=in_channels,
                          out_channels=out_channels * s_kernel_size,
                          kernel_size=1)

  def forward(self, x, A): #verilerin ağ üzerinde nasıl akacak
    x = self.conv(x)
    n, kc, t, v = x.size()
    x = x.view(n, self.s_kernel_size, kc//self.s_kernel_size, t, v)
    # Komşuluk matrisi üzerinde  Graph Convolutional yapılır ve öznitelikler eklenir.
    x = torch.einsum('nkctv,kvw->nctw', (x, A))
    return x.contiguous()

###**Zamansal Grafların Evrişimi**

Zaman grafları, graf evrişim yerine genel 2d evrişim kullanılarak uygulanabilir.
Özellik haritası (çerçeve sayısı x eklem sayısı) biçimindedir.
Çerçeve yönünde evrişim yapmak yeterli olduğu için 2d evrişim filtresi (T×1) ile uygulanabilir.

ST-GCN ayrıca uzamsal ve zamansal grafikleri dönüşümlü olarak yapar.

In [ ]:
class STGC_block(nn.Module):
  def __init__(self, in_channels, out_channels, stride, t_kernel_size, A_size, dropout=0.5):
    super().__init__()
    # Uzaysal grafiğin evrişimi
    self.sgc = SpatialGraphConvolution(in_channels=in_channels,
                                       out_channels=out_channels,
                                       s_kernel_size=A_size[0])

    # Learnable weight matrix M
    # Öğrenilebilir ağırlık matrisi M Kenarlara ağırlık verin Hangi kenarların önemli olduğunu öğrenin.
    self.M = nn.Parameter(torch.ones(A_size))

    self.tgc = nn.Sequential(nn.BatchNorm2d(out_channels),
                            nn.ReLU(),
                            nn.Dropout(dropout), # reludan once
                            nn.Conv2d(out_channels,
                                      out_channels,
                                      (t_kernel_size, 1),
                                      (stride, 1),
                                      ((t_kernel_size - 1) // 2, 0)),
                            nn.BatchNorm2d(out_channels),
                            nn.ReLU())


  def forward(self, x, A):
    x = self.tgc(self.sgc(x, A * self.M))
    return x

In [ ]:
    self.stgc1 = STGC_block(in_channels, 64, 1, t_kernel_size, A_size)
    self.stgc2 = STGC_block(64, 64, 1, t_kernel_size, A_size)
    self.stgc3 = STGC_block(64, 64, 1, t_kernel_size, A_size)
    self.stgc4 = STGC_block(64, 128, 2, t_kernel_size, A_size)
    self.stgc5 = STGC_block(128, 128, 1, t_kernel_size, A_size)
    self.stgc6 = STGC_block(128, 128, 1, t_kernel_size, A_size)
    self.stgc7 = STGC_block(128, 256, 2, t_kernel_size, A_size)
    self.stgc8 = STGC_block(256, 256, 1, t_kernel_size, A_size)
    self.stgc9 = STGC_block(256, 256, 1, t_kernel_size, A_size)
    # Prediction
    self.fc = nn.Conv2d(256, num_classes, kernel_size=1)

###**Ağ Modeli**


In [ ]:

class ST_GCN(nn.Module):
  def __init__(self, num_classes, in_channels, t_kernel_size, hop_size):
    super().__init__()
    # graf oluştur
    graph = Graph(hop_size)
    A = torch.tensor(graph.A, dtype=torch.float32, requires_grad=False)
    self.register_buffer('A', A)
    A_size = A.size()

    # Batch Normalization
    self.bn = nn.BatchNorm1d(in_channels * A_size[1])

    # STGC_blocks
    self.stgc1 = STGC_block(in_channels, 32, 1, t_kernel_size, A_size)
    self.stgc2 = STGC_block(32, 32, 1, t_kernel_size, A_size)
    self.stgc3 = STGC_block(32, 32, 1, t_kernel_size, A_size)
    self.stgc4 = STGC_block(32, 64, 2, t_kernel_size, A_size)
    self.stgc5 = STGC_block(64, 64, 1, t_kernel_size, A_size)
    self.stgc6 = STGC_block(64, 64, 1, t_kernel_size, A_size)

    # Prediction
    self.fc = nn.Conv2d(64, num_classes, kernel_size=1)

  def forward(self, x):
    # Batch Normalization
    N, C, T, V = x.size() # batch, channel, frame, node
    x = x.permute(0, 3, 1, 2).contiguous().view(N, V * C, T)
    x = self.bn(x)
    x = x.view(N, V, C, T).permute(0, 2, 3, 1).contiguous()

    # STGC_blocks
    x = self.stgc1(x, self.A)
    x = self.stgc2(x, self.A)
    x = self.stgc3(x, self.A)
    x = self.stgc4(x, self.A)
    x = self.stgc5(x, self.A)
    x = self.stgc6(x, self.A)

    # Prediction
    x = F.avg_pool2d(x, x.size()[2:])
    x = x.view(N, -1, 1, 1)
    x = self.fc(x)
    x = x.view(x.size(0), -1)
    return x

#**MODEL EĞİTİMİ**

---



In [ ]:
NUM_EPOCH = 100
BATCH_SIZE = 64

# model oluştur
model = ST_GCN(num_classes=14,
                  in_channels=3,
                  t_kernel_size=9, # Zaman grafiği evrişiminin çekirdek boyutu (t_kernel_size × 1)
                  hop_size=1).cuda()

#Optimize edici
#İlk satırda bir SGD optimizer oluşturuyoruz ve öğrenme oranını 0.01, momentumu 0.9 olarak belirliyoruz.
#Optimizerımıza sağlamamız gereken diğer bileşen ise ağımızın parametreleri bunları da veriyoruz
#model.load_state_dict(torch.load('model_weights.pth'))
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

torch.save(model.state_dict(), 'model_weights_6katman_50_1.pth')
# hata işlevi
criterion = torch.nn.CrossEntropyLoss()

# veri kümesini hazırla
data_loader = dict()
data_loader['train'] = torch.utils.data.DataLoader(dataset=Feeder(data_path='../DATASETS/cs_data/train_data.npy', label_path='../DATASETS/cs_data/train_label.npy'), batch_size=BATCH_SIZE, shuffle=True,)
data_loader['test'] = torch.utils.data.DataLoader(dataset=Feeder(data_path='../DATASETS/cs_data/test_data.npy', label_path='../DATASETS/cs_data/test_label.npy'), batch_size=BATCH_SIZE, shuffle=False)

# modeli öğrenme moduna değiştir
#model.train()

# öğrenmeye başla
for epoch in range(50, NUM_EPOCH+1):
  correct = 0
  sum_loss = 0
  for batch_idx, (data, label) in enumerate(data_loader['train']):
    data = data.cuda()
    label = label.cuda()

    output = model(data)

    loss = criterion(output, label)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    sum_loss += loss.item()
    _, predict = torch.max(output.data, 1)
    correct += (predict == label).sum().item()

  #print('# Epoch: {} | Loss: {:.4f} | Accuracy: {:.4f}'.format(epoch, sum_loss/len(data_loader['train'].dataset), (100. * correct / len(data_loader['train'].dataset))))
  print('# Epoch: {} | Loss: {:.4f} | Accuracy: {:.4f}'.format(epoch, sum_loss/len(data_loader['train']), (100. * correct / len(data_loader['train'].dataset))))

torch.save(model.state_dict(), 'model_weights_6katman_100_1.pth')

In [ ]:
data_loader = dict()
BATCH_SIZE = 64
data_loader['test'] = torch.utils.data.DataLoader(dataset=Feeder(data_path='../DATASETS/cs_data/test_data.npy', label_path='../DATASETS/cs_data/test_label.npy'), batch_size=BATCH_SIZE, shuffle=False)
# model oluştur
model = ST_GCN(num_classes=14,
                  in_channels=3,
                  t_kernel_size=9, # Zaman grafiği evrişiminin çekirdek boyutu (t_kernel_size × 1)
                  hop_size=1).cuda()
# Model yapısnı oluştur



# Kaydedilmiş ağırlıkları yükle
model.load_state_dict(torch.load('model_weights_9katman_200.pth'))
sum(p.numel() for p in model.parameters() if p.requires_grad)

245814

#**MODEL DEĞERLENDİRMESİ**



In [ ]:
#Modeli değerlendirme moduna değiştir

data_loader = dict()
BATCH_SIZE = 64
data_loader['test'] = torch.utils.data.DataLoader(dataset=Feeder(data_path='../DATASETS/cs_data/test_data.npy', label_path='../DATASETS/cs_data/test_label.npy'), batch_size=BATCH_SIZE, shuffle=False)

# Model yapısnı oluştur
model = ST_GCN(num_classes=14,
                  in_channels=3,
                  t_kernel_size=9, # Zaman grafiği evrişiminin çekirdek boyutu (t_kernel_size × 1)
                  hop_size=1)


# Kaydedilmiş ağırlıkları yükle
#model.load_state_dict(torch.load('model_weights_6katman_50_1.pth'))
model.load_state_dict(torch.load('model_weights_6katman_50_1.pth', map_location=torch.device('cpu')))

# Modeli değerlendirme moduna al (eğer modeli tahmin yapmak için kullanacaksanız)
model.eval()

correct = 0
confusion_matrix = np.zeros((14, 14))
with torch.no_grad():
  for batch_idx, (data, label) in enumerate(data_loader['test']):
    #data = data.cuda()
    #label = label.cuda()

    output = model(data)

    _, predict = torch.max(output.data, 1)
    correct += (predict == label).sum().item()

    for l, p in zip(label.view(-1), predict.view(-1)):
      confusion_matrix[l.long(), p.long()] += 1

len_cm = len(confusion_matrix)

total_sum = np.sum(confusion_matrix)
print(total_sum)

for i in range(len_cm):
    sum_cm = np.sum(confusion_matrix[i])
    for j in range(len_cm):
        confusion_matrix[i][j] = 100 * (confusion_matrix[i][j] / sum_cm)


classes = ['pick up','sit down','stand up','put on jacket',
           'take off jacket','put on a shoe','put on glasses','take off glasses',
           'put on a hat/cap','take off a hat/cap','cheer up','hand waving',
           'hopping', 'jump up']

plt.imshow(confusion_matrix, interpolation='nearest', cmap=plt.cm.Blues)
plt.colorbar()


plt.title('Confusion matrix')
plt.tight_layout()

tick_marks = np.arange(len(classes))
plt.xticks(tick_marks, classes, rotation=90)
plt.yticks(tick_marks, classes)
plt.ylabel('True')
plt.xlabel('Predicted')

plt.show()


print('# Test Accuracy: {:.3f}[%]'.format(100. * correct / len(data_loader['test'].dataset)))